## Error Analysis

In [192]:
import pandas as pd
import joblib

In [193]:
df_svm = pd.read_csv('/Users/liki/neuefische/Respiratory_Disease_Classification/data/misclassified_svm.csv')
df_resnet = pd.read_csv('/Users/liki/neuefische/Respiratory_Disease_Classification/data/misclassified_testset_ResNet.csv')

In [194]:
df_svm['filename'] = df_svm['filename'].apply(lambda x: x[2:6])
df_svm['filename'] = df_svm['filename'].apply(lambda x: x.replace('_',''))
df_svm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  319 non-null    int64 
 1   filename    319 non-null    object
 2   actual      319 non-null    int64 
 3   pred        319 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 10.1+ KB


In [195]:
df_svm['filename'].value_counts()

i194    34
i224    34
i144    31
i171    17
i102    17
i136    17
i123    17
i126    16
i202    16
i214    16
i143    14
i187    11
f85     11
i225     9
f99      7
i182     6
f82      6
f72      5
f63      5
f80      4
f69      4
f101     3
f73      3
f64      3
i223     2
f55      2
i109     2
i185     1
i150     1
f87      1
i195     1
i211     1
f75      1
i160     1
Name: filename, dtype: int64

In [196]:
file_svm = df_svm['filename'].unique()
file_svm.shape

(34,)

In [197]:
df_resnet['filename'] = df_resnet['filename'].apply(lambda x: x[4:8])
df_resnet['filename'] = df_resnet['filename'].apply(lambda x: x.replace('\'',''))

df_resnet.tail(20)
file_rn = df_resnet['filename'].unique()
file_rn.shape

(42,)

In [198]:
df_resnet.filename.value_counts()

i224    31
i194    31
i144    24
i202    16
i195    14
i185    13
i109    12
i214    11
i175     9
f64      9
f69      9
i104     9
i223     9
i126     8
i225     8
i171     7
i211     7
i113     7
i124     6
i118     6
i182     6
i102     5
i149     4
f63      4
i160     3
i216     3
f23      3
f55      3
f29      3
i169     3
i143     3
i101     2
i187     2
f101     2
i136     2
f85      2
f72      2
i205     2
i150     2
i165     1
i178     1
f99      1
Name: filename, dtype: int64

In [199]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

print(len(intersection(file_rn, file_svm)))
intersection(file_rn, file_svm)

28


['f101',
 'f55',
 'f63',
 'f64',
 'f69',
 'i109',
 'i150',
 'i160',
 'i185',
 'i195',
 'i211',
 'i223',
 'f72',
 'f85',
 'f99',
 'i102',
 'i126',
 'i136',
 'i143',
 'i144',
 'i171',
 'i182',
 'i187',
 'i194',
 'i202',
 'i214',
 'i224',
 'i225']

In [200]:
df_anno = pd.read_csv('/Users/liki/neuefische/Respiratory_Disease_Classification/data/annotations.csv')
df_anno.head()

,Unnamed: 0,id,diagnosis,train_test,filename,is_healthy
0,0,i101,urti,test,101_1b1_Al_sc_Meditron.wav,0
1,1,i101,urti,test,101_1b1_Pr_sc_Meditron.wav,0
2,2,i102,healthy,test,102_1b1_Ar_sc_Meditron.wav,1
3,3,i103,asthma,train,103_2b2_Ar_mc_LittC2SE.wav,0
4,4,i104,copd,test,104_1b1_Al_sc_Litt3200.wav,0


In [201]:
df_anno[df_anno.train_test=='test'].id.nunique()

57

In [222]:
data_test = joblib.load('spectrogram_test_80x80px.pkl')
files = pd.DataFrame(data_test["filename"], columns = ["filename"])
files

,filename
0,1_f75_4496.png
1,1_f75_4482.png
2,1_f82_4706.png
3,1_f75_4469.png
4,1_f87_4917.png
...,...
1627,0_i178_2257.png
1628,0_i176_2100.png
1629,0_i176_2114.png
1630,0_i178_2243.png


In [223]:
# files["filename"] = files["filename"].apply(lambda x: x[2:6])
# files["filename"] = files["filename"].apply(lambda x: x.replace('_',''))

files

,filename
0,f75
1,f75
2,f82
3,f75
4,f87
...,...
1627,i178
1628,i176
1629,i176
1630,i178


In [231]:
chunks_re = pd.DataFrame(df_resnet.filename.value_counts())
chunks_re.columns = ['resnet']
chunks_svm = pd.DataFrame(df_svm.filename.value_counts())
chunks_svm.columns = ['svm']
chunks_tot = pd.DataFrame(files.filename.value_counts())
chunks_tot.columns = ['tot']

In [243]:
err_resnet = chunks_re.join(chunks_tot)
err_resnet['perc'] = err_resnet['resnet']/err_resnet['tot'] * 100
err_resnet.sort_values(by='perc', ascending=False)

,resnet,tot,perc
i202,16,16,100.000000
i224,31,34,91.176471
i194,31,34,91.176471
f64,9,12,75.000000
f69,9,12,75.000000
i144,24,33,72.727273
i214,11,16,68.750000
i150,2,3,66.666667
i185,13,20,65.000000
i109,12,20,60.000000


In [242]:
err_svm = chunks_svm.join(chunks_tot)
err_svm['perc'] = err_svm['svm']/err_svm['tot'] * 100
err_svm.sort_values(by='perc', ascending=False)

,svm,tot,perc
i194,34,34,100.000000
i224,34,34,100.000000
i171,17,17,100.000000
i102,17,17,100.000000
i136,17,17,100.000000
i123,17,17,100.000000
i202,16,16,100.000000
i214,16,16,100.000000
i126,16,17,94.117647
i144,31,33,93.939394


i202 100% misclassification rate
i224
i194
i214
